# Clone le repository pubic GIT pour récupérer notre portefuille BASEAUTO.csv 

In [ ]:
!rm -rf ActuarialThesis
!git clone https://github.com/aderdouri/ActuarialThesis.git
%ls -ltr ActuarialThesis

# Importer les librairies de base Python et sklearn pour le preprocessing du dataset.

In [ ]:
# The basics
import pandas as pd
import numpy as np

# Data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv('ActuarialThesis/Data/BASEAUTO.csv')

In [ ]:
df.columns

## Encoding des différentes variables de notre dataset

Les deux variables `PERMIS` et `AGECOND` présentent une forte correlation, je supprime la variable `PERMIS`.


## La variable `PERMIS`

In [ ]:
df.drop('PERMIS', axis=1, inplace=True)

## La variable `USAGE`

La variable `USAGE` est bien évidemment une variables catégorique, et nous devons les transformons ainsi.

In [ ]:
df['USAGE'].unique()

In [ ]:
# USAGE (1: Privé, 2: Privé-Trajet, 3: Professionnel, 4: Tournées)
def USAGETransform(usage):
  if usage==1:
    return 'Prive'
  elif usage==2:
    return 'PriveTrajet'
  elif usage==3:
    return 'Professionnel'
  elif usage==4:
    return 'Tournees'

In [ ]:
df['USAGE'] = df['USAGE'].apply(USAGETransform)

In [ ]:
df['USAGE'].unique()

 ## La variable `GARAGE`

La variable GARAGE est bien évidemment une variables catégorique, et nous devons les transformons ainsi.

In [ ]:
df['GARAGE'].unique()

In [ ]:
# GARAGE (1: Individuel clos, 2: Clos collectif, 3: Autre cas)
def GARAGETransform(garage):
  if garage==1:
    return 'IndividuelClos'
  elif garage==2:
    return 'ClosCollectif'
  elif garage==3:
    return 'GarageAutre'

In [ ]:
df['GARAGE'] = df['GARAGE'].apply(GARAGETransform)

In [ ]:
df['GARAGE'].unique()

## La variable `CSP`

In [ ]:
100*df['CSP'].value_counts(normalize=True)

Plus que 57\% des observations ont une CSP égale à 50, soit on supprime cette variable, soit on la transforme de la manière suivante: on garde les catégories 50 et 60 et on regroupe toutes les autres catégries en une seule catégrie nommée `CSP_AUTRE`. En suite on applique une transformation One-Hot-Encoding sur cette variable. On peut revenir sur ce split et l'affiner si lors de la phase d'apprentissage, la caractéristique `CSP` s'avère importante.

In [ ]:
# Transformation de la variable CSP
df['CSP'] = df['CSP'].apply(lambda CSP : 'CSP_' + str(CSP) if CSP in [50, 60] else 'CSP_AUTRE')

In [ ]:
df['CSP'].unique()

## La variable `CAR`

In [ ]:
100*df['CAR'].value_counts(normalize=True)

Plus que 70% des observations ont la variable CAR à BERLING, soit on supprime cette variable, soit on la transforme de la manière suivante: On la transforme en une variable binaire qui prend la valeur `CAR_BERLINE` si BER sinon `CAR_AUTRE`.

In [ ]:
df['CAR'] = df['CAR'].map(lambda val: 'CAR_BERLINE' if val=='BER' else 'CAR_AUTRE')

In [ ]:
df.head()

On n'opère aucune transformation supplémentaire à ce stade et on sauvegarde le dataset ainsi.

In [ ]:
df.to_csv('ActuarialThesis/Data/transformedBASEAUTO.csv', index=False)

In [ ]:
# Restaurer le dadaset ainsi sauvegardé.
transformed_df = pd.read_csv('ActuarialThesis/Data/transformedBASEAUTO.csv')
transformed_df.head()

In [ ]:
!ls -ltr ActuarialThesis/Data

In [ ]:
transformed_df.info()

Egalement, on considère un autre encoding pour les variables `AGECOND` et `ACV`.
* La caractéristique `AGECOND` sera scindé en plusieurs groupes:
`(18,21], (21,25], (25,35], (35,45], (45,55], (55,70], (70, ].`
* De même la caractéristique `ACV` sera scindé en groupes : `(0,1], (1,4], (4,10], (10,].`

On distingue les deux cas suivants et considérer entrainer nos modèles sur les différentes transformations pour pouvoir les comparer:
- Cas 1: Les caractéristiques `AGECOND` et `ACV` sont considérés comme des variables continues.
- Cas 2: Les caractéristiques `AGECOND` et `ACV` sont considérés comme des  variables catégorielles.

In [ ]:
def AGETransform(age):
  if age <=21:
    return '<21'
  elif age >21 and age <=25:
    return '21_25'
  elif age >25 and age <=35:
    return '25_35'    
  elif age >35 and age <=45:
    return '35_45'    
  elif age >45 and age <=55:
    return '45_55'    
  elif age >55 and age <=70:
    return '55_70'        
  elif age >70:
    return '>70'

In [ ]:
df['AGECOND_CAT'] = df['AGECOND'].apply(AGETransform)

In [ ]:
df['AGECOND_CAT'].unique()

In [ ]:
def ACVTransform(acv):
  if acv <=1:
    return '<=1'
  elif acv >1 and acv <=4:
    return '1_4'
  elif acv >4 and acv <=10:
    return '4_10'        
  elif acv >10:
    return '>10'

In [ ]:
df['ACV_CAT'] = df['ACV'].apply(ACVTransform)

In [ ]:
df['ACV_CAT'].unique()

In [ ]:
cat_cols = ['USAGE', 'CLA', 'ALI', 'ENE', 'VIT', 'SEGM', 'GARAGE', 'CSP', 'AGECOND_CAT', 'ACV_CAT']

In [ ]:
bin_cols = ['SEX', 'STATUT', 'K8000', 'CAR']

In [ ]:
num_cols = ['AGECOND', 'ACV', 'RM']

In [ ]:
# Separating target from dataset 
Y = df['CHARGE']
X = df.drop('CHARGE', axis=1)

# Mapping et encoding de l'ensemble des variables.

In [ ]:
newX = X.copy()
newX.head()

In [ ]:
# Mapping SEX
sex_map = {'M': 0,
           'F': 1}
newX.SEX = newX.SEX.map(sex_map)

In [ ]:
# Mapping STATUT
statut_map = {'A': 0,
              'C': 1}
newX.STATUT = newX.STATUT.map(statut_map)

In [ ]:
# Mapping K8000
k8000_map = {'N': 0,
             'O': 1}
newX.K8000 = newX.K8000.map(k8000_map)

In [ ]:
# Mapping CAR
CAR_map = {'CAR_BERLINE': 0,
             'CAR_AUTRE': 1}
newX.CAR = newX.CAR.map(CAR_map)

In [ ]:
newX.head()

In [ ]:
# Appliquer le OneHotEncding à l'ensemble des variables catégoriques
ohe = OneHotEncoder()
ohe.fit(newX[cat_cols])
transformed = ohe.transform(newX[cat_cols]).toarray()
ohe_df = pd.DataFrame(transformed, columns=ohe.get_feature_names_out())
# Concat with original data
newX = pd.concat([newX, ohe_df], axis=1).drop(cat_cols, axis=1)

In [ ]:
newX.head(5)

In [ ]:
newX.columns

In [ ]:
# Appliquer le StandarScaling à l'ensemble des variables numériques.
scaler = StandardScaler()
scaler.fit(X[num_cols])
scaledx = scaler.transform(X[num_cols])

In [ ]:
scaledx

In [ ]:
scaledx_df = pd.DataFrame(data = scaledx, columns = num_cols)
scaledx_df

In [ ]:
newX.head()

In [ ]:
newX.AGECOND = scaledx_df.AGECOND
newX.ACV = scaledx_df.ACV
newX.RM = scaledx_df.RM
newX.head()

## Sauvegarder le dataset ainsi transformé (newX, Y) séparément.

- Cas 1: Les caractéristiques `AGECOND` et `ACV`
 sont considérés comme des variables continues

In [ ]:
AGECOND_CAT_cols = [col for col in newX if col.startswith('AGECOND_')]
AGECOND_CAT_cols

In [ ]:
ACV_CAT_cols = [col for col in newX if col.startswith('ACV_')]
ACV_CAT_cols

In [ ]:
newX_CONT = newX.drop(AGECOND_CAT_cols, axis=1)
newX_CONT = newX_CONT.drop(ACV_CAT_cols, axis=1)

In [ ]:
# Sauvegarder le dataset ainsi transformé.
encoded_df = pd.concat([newX_CONT, Y], axis=1)
encoded_df.to_csv('ActuarialThesis/Data/encodedBASEAUTO.csv', index=False)

In [ ]:
# Restaurer le dadaset ainsi sauvegardé.
encoded_df = pd.read_csv('ActuarialThesis/Data/encodedBASEAUTO.csv')
encoded_df.head()

In [ ]:
encoded_df.info()

Cas 2: Les caractéristiques `AGECOND` et `ACV` sont considérés comme des variables catégorielles.

In [ ]:
newX_CAT = newX.drop(['AGECOND', 'ACV'], axis=1)

In [ ]:
# Sauvegarder le dataset ainsi transformé.
encoded_categorical_df = pd.concat([newX_CAT, Y], axis=1)
encoded_categorical_df.to_csv('ActuarialThesis/Data/encodedCategoricalBASEAUTO.csv', index=False)

In [ ]:
# Restaurer le dadaset ainsi sauvegardé.
new_encoded_categorical_df = pd.read_csv('ActuarialThesis/Data/encodedCategoricalBASEAUTO.csv')
new_encoded_categorical_df.head()

In [ ]:
new_encoded_categorical_df.info()

In [ ]:
new_encoded_categorical_df.columns